In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from skimage import io
import torch
from glob import glob
from torchsummary import summary
import os
import shutil
import time

In [2]:
data_dir=r'C:\Users\win10\Desktop\6207\BS6207-assignment4'
artifacts_dir="artifacts"
cancer_regions='cancer_regions'
normal_regions='normal_regions'
other='other'

In [3]:
train_dir=os.path.join(data_dir,'new','trainSet')
valid_dir=os.path.join(data_dir,'new','validSet')
test_dir=os.path.join(data_dir,'new','testSet')
test_dir

'C:\\Users\\win10\\Desktop\\6207\\BS6207-assignment4\\new\\testSet'

In [40]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
train_transform=transforms.Compose([
    transforms.ToTensor(),
])

test_transform=transforms.Compose([
    transforms.ToTensor(),
])

#generate dataloader
train_ds=ImageFolder(train_dir,transform=train_transform)
valid_ds=ImageFolder(valid_dir,transform=test_transform)
test_ds=ImageFolder(test_dir,transform=test_transform)

In [41]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_ds, batch_size=batch_size, shuffle=True)

In [13]:
import numpy as np
import torch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')	# 这里会存储迄今最优模型的参数
        self.val_loss_min = val_loss

In [80]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.convs = torch.nn.Sequential(
            # Defining a 2D convolution layer
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            #BatchNorm2d(4),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            #BatchNorm2d(4),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            #BatchNorm2d(4),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            #BatchNorm2d(4),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            #BatchNorm2d(4),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fully_c = torch.nn.Sequential(
            torch.nn.Linear(512 * 4 * 4, 4096),
            torch.nn.Dropout(0.1),
            torch.nn.ReLU(),
            torch.nn.Linear(4096, 1024),
            torch.nn.Dropout(0.1),
            torch.nn.ReLU(),
            torch.nn.Linear(1024,4),
        )
        self.flatten = torch.nn.Flatten()
    # Defining the forward pass    
    def forward(self, x):
        x = self.convs(x)
        x = self.flatten(x)
        x = self.fully_c(x)
        return x

In [88]:
model=Model()
summary(model, (3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           1,792
       BatchNorm2d-2           [-1, 64, 64, 64]             128
              ReLU-3           [-1, 64, 64, 64]               0
         MaxPool2d-4           [-1, 64, 32, 32]               0
            Conv2d-5          [-1, 128, 16, 16]          73,856
       BatchNorm2d-6          [-1, 128, 16, 16]             256
              ReLU-7          [-1, 128, 16, 16]               0
         MaxPool2d-8            [-1, 128, 8, 8]               0
            Conv2d-9            [-1, 256, 4, 4]         295,168
      BatchNorm2d-10            [-1, 256, 4, 4]             512
             ReLU-11            [-1, 256, 4, 4]               0
        MaxPool2d-12            [-1, 256, 2, 2]               0
          Flatten-13                 [-1, 1024]               0
           Linear-14                  [

In [50]:
def train(net,train_iter,valid_iter,optim,epoches,early_stop=False,name=None):
    history_loss=dict()
    history_acc=dict()
    history_loss["val"]=[]
    history_loss["train"]=[]
        
    history_acc["val"]=[]
    history_acc["train"]=[]
    best_acc=0
    for epoch in range(epoches):
        print("processing the {} epoch".format(epoch))
        epoch_start_time = time.time()
        train_acc = []
        train_loss = []
        val_acc = []
        val_loss = []
        len_train=0
        net.train()
        for batch_x,batch_y in train_iter:
            y_hat=net(batch_x)
            l=loss(y_hat,batch_y)
            optim.zero_grad()
            l.backward()
            optim.step()
            train_acc.append(np.sum(np.argmax(y_hat.detach().numpy(),axis=1)==batch_y.detach().numpy()))
            train_loss.append(l.item())
            len_train+=batch_x.shape[0]

        train_loss=np.sum(train_loss)/len_train
        train_acc=np.sum(train_acc)/len_train
        history_loss["train"].append(train_loss)
        history_acc["train"].append(train_acc)
        #eval
        net.eval()
        
        len_val=0
        with torch.no_grad():
            for batch_x,batch_y in valid_iter:
                y_hat=net(batch_x)
                l=loss(y_hat,batch_y)
                val_acc.append(np.sum(np.argmax(y_hat.numpy(),axis=1)==batch_y.numpy()))
                val_loss.append(l.item())
                len_val+=batch_x.shape[0]
            
        val_loss=np.sum(val_loss)/len_val
        val_acc=np.sum(val_acc)/len_val
        history_loss["val"].append(val_loss)
        history_acc["val"].append(val_acc)
        print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % \
                (epoch + 1, epoches, time.time()-epoch_start_time, \
                 train_acc, train_loss, val_acc, val_loss))
        
        early_stopping(val_loss, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break

    return history_loss,history_acc

In [46]:
def test(net,test_iter):
    
    for epoch in range(1):
        print("processing the {} epoch".format(epoch))
        epoch_start_time = time.time()
        #eval
        net.eval()
        test_acc=[]
        test_loss=[]
        len_test=0
        with torch.no_grad():
            for batch_x,batch_y in test_iter:
                y_hat=net(batch_x)
                l=loss(y_hat,batch_y)
                test_acc.append(np.sum(np.argmax(y_hat.numpy(),axis=1)==batch_y.numpy()))
                test_loss.append(l.item())
                len_test+=batch_x.shape[0]
            
        test_loss=np.sum(test_loss)/len_test
        test_acc=np.sum(test_acc)/len_test
        print('[%03d/%03d] %2.2f sec(s) Test Acc: %3.6f Loss: %3.6f' % \
                (epoch + 1, 1, time.time()-epoch_start_time, \
                 test_acc, test_loss))

In [47]:
lr=0.01
dropout=0.3

In [89]:
model=Model()
optim=torch.optim.Adam(model.parameters(),lr=lr)
loss=torch.nn.CrossEntropyLoss()

In [90]:
his_loss,his_acc=train(model, train_loader, valid_loader, optim, 100, early_stop=False,name="model_normal")

processing the 0 epoch
[001/100] 50.58 sec(s) Train Acc: 0.623541 Loss: 0.019073 | Val Acc: 0.711656 loss: 0.007472
EarlyStopping counter: 5 out of 10
processing the 1 epoch
[002/100] 53.71 sec(s) Train Acc: 0.700287 Loss: 0.005842 | Val Acc: 0.720092 loss: 0.005422
EarlyStopping counter: 6 out of 10
processing the 2 epoch
[003/100] 59.28 sec(s) Train Acc: 0.718086 Loss: 0.005394 | Val Acc: 0.735429 loss: 0.005343
Validation loss decreased (0.005376 --> 0.005343).  Saving model ...
processing the 3 epoch
[004/100] 49.22 sec(s) Train Acc: 0.760957 Loss: 0.004756 | Val Acc: 0.711656 loss: 0.008188
EarlyStopping counter: 1 out of 10
processing the 4 epoch
[005/100] 40.90 sec(s) Train Acc: 0.794067 Loss: 0.004226 | Val Acc: 0.373466 loss: 0.013223
EarlyStopping counter: 2 out of 10
processing the 5 epoch
[006/100] 38.93 sec(s) Train Acc: 0.823732 Loss: 0.003813 | Val Acc: 0.611963 loss: 0.018876
EarlyStopping counter: 3 out of 10
processing the 6 epoch
[007/100] 40.88 sec(s) Train Acc: 0.8

In [91]:
test(model,test_loader)

processing the 0 epoch
[001/001] 1.74 sec(s) Test Acc: 0.680000 Loss: 0.010617
